<a href="https://colab.research.google.com/github/yifan-zhou19/Policy_migration/blob/master/Video_matrix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#define ABAC policy (video)


U_v=[
   
{'uid':'child1r','ageGroup':'child','memberType':'regular'},
{'uid':'child2r','ageGroup':'child','memberType':'regular'},
{'uid':'teen1r','ageGroup':'teen','memberType':'regular'},
{'uid':'teen2r','ageGroup':'teen','memberType':'regular'},
{'uid':'adult1r','ageGroup':'adult','memberType':'regular'},
{'uid':'adult2r','ageGroup':'adult','memberType':'regular'}, 

{'uid':'child1p','ageGroup':'child','memberType':'premium'},
{'uid':'child2p','ageGroup':'child','memberType':'premium'},
{'uid':'teen1p','ageGroup':'teen','memberType':'premium'},
{'uid':'teen2p','ageGroup':'teen','memberType':'premium'},
{'uid':'adult1p','ageGroup':'adult','memberType':'premium'},
{'uid':'adult2p','ageGroup':'adult','memberType':'premium'},  
]

R_v=[

{'rid':'the_lion_king','rating':'G','videoType':'old'},
{'rid':'toy_story','rating':'G','videoType':'old'},
{'rid':'star_wars','rating':'PG','videoType':'old'},
{'rid':'jaws','rating':'PG','videoType':'old'},
{'rid':'brave_heart','rating':'R','videoType':'old'},
{'rid':'my_cousin_vinny','rating':'R','videoType':'old'},


{'rid':'happy_feet','rating':'G','videoType':'new'},
{'rid':'happy_feet_2','rating':'G','videoType':'new'},
{'rid':'cars','rating':'G','videoType':'new'},
{'rid':'brave','rating':'PG','videoType':'new'},
{'rid':'life_of_pi','rating':'PG','videoType':'new'},
{'rid':'flight','rating':'R','videoType':'new'},
{'rid':'cloud_atlas','rating':'R','videoType':'new'}   
]
UA_v=['ageGroup','memberType','uid']
RA_v=['rid','rating','videoType']
O_v=['view']
RU_v=[
  
# premium members of all ages can view movies rated G.
{'expressions':[{'eu':{'memberType':[('is','premium')]},'er':{'raing':[('is','G')]}}],'operations':['view'],'decision':'allow'}, 
# premium teens can view movies rated PG.
{'expressions':[{'eu':{'memberType':[('is','premium')],'ageGroup':[('is','teen')]},'er':{'raing':[('is','PG')]}}],'operations':['view'],'decision':'allow'}, 
# premium adults can view movies with all ratings.
{'expressions':[{'eu':{'memberType':[('is','premium')],'ageGroup':[('is','adult')]},'er':{}}],'operations':['view'],'decision':'allow'}, 
# members of all ages can view old movies rated G.
{'expressions':[{'eu':{},'er':{'raing':[('is','G')],'videoType':[('is','old')]}}],'operations':['view'],'decision':'allow'}, 

# teens can view old movies rated PG.
{'expressions':[{'eu':{'ageGroup':[('is','teen')]},'er':{'raing':[('is','PG')],'videoType':[('is','old')]}}],'operations':['view'],'decision':'allow'}, 
# adults can view old movies with all ratings.
{'expressions':[{'eu':{'ageGroup':[('is','adult')]},'er':{'videoType':[('is','old')]}}],'operations':['view'],'decision':'allow'}, 

]
def dU(user,attribute):
  if attribute in user.keys():
    return user[attribute]

def dR(resource,attribute):
  if attribute in resource.keys():
    return resource[attribute]
ABAC_video=(U_v,R_v,O_v,UA_v,RA_v,dU,dR,RU_v)

In [3]:
def satisfied(u,r,expressions):
  for expression in expressions:
    eu=expression['eu']
    er=expression['er']
    #for all the keys in eu get the constrina of this key
    #if u does not satisfy this const return false
    for key in eu.keys():
      for (op,c) in eu[key]:
        #op,c=eu[key]
        #print(op,c)
        if op == 'is':
          if dU(u,key) != c:
            return False

    #for all the keys in er get the constrinat of the key
    for key in er.keys():
      for (op,c) in er[key]:
        #print(op,c)
        #op,c=er[key]
        if(dR(r,key)==None):
          return False
        if op =='is':
          if dR(r,key)!=c:
        #  print(dR(r,key),c)
            return False
        elif op == '=':
          if dR(r,key)!=dU(u,c) and dR(r,key)!=dR(r,c):
            return False
        elif op == 'in':
          if dU(u,c)==None and dR(r,c)==None:
            return False
          if type(dU(u,c)==list) and dU(u,c)!=None:
          #print(dU(u,c))
            if dR(r,key) not in dU(u,c):
              return False
          if type(dR(r,c)==list) and dR(r,c)!=None:
          #print(dU(u,c))
            if dR(r,key) not in dR(r,c):
              return False
  
  return True


def check_ABAC_policy(request,abac_policy):
  u,r,o =request
  _,_,_,_,_,_,_,rules = abac_policy
  for rule in rules:
    if o in rule['operations']:
      if satisfied(u,r,rule['expressions']):
        return True
  return False

# check the ABAC policy 
number_allow=0
number_deny=0
for u in U_v:
  for r in R_v:
    for o in O_v:
      if(check_ABAC_policy((u,r,o),ABAC_video)==True):
        number_allow= number_allow+1
      else:
        number_deny =number_deny +1 
        #print(f'{u}\n{r}\n{o}\n \n')

print(f'For ABAC policy:\nNumber of allow:{number_allow}\nNumber of deny:{number_deny}')

#print(f'{check_ABAC_policy((U[6],R[7],"readMyScores"),ABAC_university)}\n{U[6]}\n{R[7]}')
#print(f'{check_ABAC_policy((U[3],R[5],"addScore"),ABAC_university)}\n{U[3]}\n{R[5]}')
#print(f'Loss at epo {epoch}: {losses/len(training_data)}')



For ABAC policy:
Number of allow:38
Number of deny:118


In [4]:
from numpy import *
# Calculate the number of user constraints
# Calculate the number of permissions(resource constraint,operation)
UC=[]
Permission=[]
for rule in RU_v:
  user_cons=rule['expressions'][0]['eu']
  resource_cons=rule['expressions'][0]['er']
  operations=rule['operations']
  for uc_key in user_cons.keys():
    uc = {uc_key:user_cons[uc_key]}
    if uc not in UC:
      UC.append(uc)
  if (resource_cons,operations) not in Permission:
    Permission.append((resource_cons,operations))
n_uc=len(UC)
n_p=len(Permission)
print(f'Number of user constraints:{n_uc}\nNumber of Permission sets:{n_p}')
print(UC)
print(Permission)


Number of user constraints:3
Number of Permission sets:6
[{'memberType': [('is', 'premium')]}, {'ageGroup': [('is', 'teen')]}, {'ageGroup': [('is', 'adult')]}]
[({'raing': [('is', 'G')]}, ['view']), ({'raing': [('is', 'PG')]}, ['view']), ({}, ['view']), ({'raing': [('is', 'G')], 'videoType': [('is', 'old')]}, ['view']), ({'raing': [('is', 'PG')], 'videoType': [('is', 'old')]}, ['view']), ({'videoType': [('is', 'old')]}, ['view'])]


In [5]:
def get_item_id(item,list):
  id = 0
  while(id<len(list)):
    if item == list[id]:
      return id
    id = id + 1
  return None

def cover(ma,mb):# ma cover mb
  if ma.shape[1] != mb.shape[1]:
    return False
  for i in range(ma.shape[1]):
    if mb[0,i]==1 and ma[0,i]==0:
      return False


  return True

In [6]:
# permission-constraint matrix
pc=mat(zeros((n_p,n_uc)))
for rule in RU_v:
  user_cons=rule['expressions'][0]['eu']
  resource_cons=rule['expressions'][0]['er']
  operations=rule['operations']
  p_id=get_item_id((resource_cons,operations),Permission)
  for uc_key in user_cons.keys():
    uc = {uc_key:user_cons[uc_key]}
    c_id=get_item_id(uc,UC)
    if(c_id==None):
      continue
    pc[p_id,c_id]=1
print(f'The pc matrix is\n {pc}')
rp=mat(diag(array(ones(n_p))))
for i in range(n_p-1):
  for j in range(i+1,n_p):
    if (cover(pc[i,],pc[j,])):
      rp[i,j]=1
    if (cover(pc[j,],pc[i,])):
      rp[j,i]=1

print(f'The rp matrix is\n {rp}')

The pc matrix is
 [[1. 0. 0.]
 [1. 1. 0.]
 [1. 0. 1.]
 [0. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
The rp matrix is
 [[1. 0. 0. 1. 0. 0.]
 [1. 1. 0. 1. 1. 0.]
 [1. 0. 1. 1. 0. 1.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 1. 0.]
 [0. 0. 0. 1. 0. 1.]]


In [19]:
def rest_of(l,r):
  vector = mat(zeros((1,n_uc)))
  for role in l:
    if role != r:
      vector = vector + rc[role,]
  for i in range(n_uc):
    if vector[0,i]>0:
      vector[0,i]=1
  return vector

import copy
# User role assignment 
n_r=n_p
rc=pc
URA = dict()
for u in U_v:
  URA[u['uid']]=[]
  uv=mat(ones((1,n_uc)))
  for cid in range(n_uc):
    expressions=[{'eu':UC[cid],'er':{}}]
    if satisfied(u,None,expressions)==False:
      uv[0,cid]=0
  for rid in range(n_r):
    if cover(uv,rc[rid,]):
      URA[u['uid']].append(rid)

# remove hierarchy
  for ri in URA[u['uid']]:
    rest = URA[u['uid']].copy()
    rest.remove(ri)

    for rj in rest:
      if cover(rc[rj,],rc[ri,]) and ri in URA[u['uid']]:
        URA[u['uid']].remove(ri)
#remove redundant
  for r in URA[u['uid']]:
    if len(URA[u['uid']])==1:
      break
    if cover(rest_of(URA[u['uid']],r),rc[r,]):
      URA[u['uid']].remove(r)

  ura =  URA[u['uid']]
  print(f'User {u}, uv {uv}, URA {ura}')


User {'uid': 'child1r', 'ageGroup': 'child', 'memberType': 'regular'}, uv [[0. 0. 0.]], URA [3]
User {'uid': 'child2r', 'ageGroup': 'child', 'memberType': 'regular'}, uv [[0. 0. 0.]], URA [3]
User {'uid': 'teen1r', 'ageGroup': 'teen', 'memberType': 'regular'}, uv [[0. 1. 0.]], URA [4]
User {'uid': 'teen2r', 'ageGroup': 'teen', 'memberType': 'regular'}, uv [[0. 1. 0.]], URA [4]
User {'uid': 'adult1r', 'ageGroup': 'adult', 'memberType': 'regular'}, uv [[0. 0. 1.]], URA [5]
User {'uid': 'adult2r', 'ageGroup': 'adult', 'memberType': 'regular'}, uv [[0. 0. 1.]], URA [5]
User {'uid': 'child1p', 'ageGroup': 'child', 'memberType': 'premium'}, uv [[1. 0. 0.]], URA [0]
User {'uid': 'child2p', 'ageGroup': 'child', 'memberType': 'premium'}, uv [[1. 0. 0.]], URA [0]
User {'uid': 'teen1p', 'ageGroup': 'teen', 'memberType': 'premium'}, uv [[1. 1. 0.]], URA [1]
User {'uid': 'teen2p', 'ageGroup': 'teen', 'memberType': 'premium'}, uv [[1. 1. 0.]], URA [1]
User {'uid': 'adult1p', 'ageGroup': 'adult', 'me

In [20]:
# check RBAC policy
def check_RBAC_policy(request,rp,URA):
  u,r,o =request
  for role_id in URA[u['uid']]:
    for pid in range(n_p):
      if rp[role_id,pid]==1:
        (er,operations) = Permission[pid]
        expressions=[{'eu':{},'er':er}]
        if satisfied(None,r,expressions) and o in operations:
          return True


  return False


In [21]:
# Check the RBAC
# check the RBAC policy 

number_allow=0
number_deny=0
for u in U_v:
  for r in R_v:
    for o in O_v:
      if(check_RBAC_policy((u,r,o),rp,URA)==True):
        number_allow= number_allow+1
      else:
        number_deny =number_deny +1 
        #print(f'{u}\n{r}\n{o}\n \n')

print(f'For RBAC:\nNumber of allow:{number_allow}\nNumber of deny:{number_deny}')

For RBAC:
Number of allow:38
Number of deny:118


In [23]:

# check the RBAC policy 
number_allow=0
number_deny=0
for u in U_v:
  for r in R_v:
    for o in O_v:
      if(check_RBAC_policy((u,r,o),rp,URA)==True):
        number_allow= number_allow+1
      else:
        number_deny =number_deny +1 
        #print(f'{u}\n{r}\n{o}\n \n')

print(f'For RBAC:\nNumber of allow:{number_allow}\nNumber of deny:{number_deny}')

# check the ABAC policy 
number_allow=0
number_deny=0
for u in U_v:
  for r in R_v:
    for o in O_v:
      if(check_ABAC_policy((u,r,o),ABAC_video)==True):
        number_allow= number_allow+1
      else:
        number_deny =number_deny +1 
        #print(f'{u}\n{r}\n{o}\n \n')

print(f'For ABAC:\nNumber of allow:{number_allow}\nNumber of deny:{number_deny}')


#number_allow=0
#number_deny=0

correct=0
for u in U_v:
  for r in R_v:
    for o in O_v:
      if (check_ABAC_policy((u,r,o),ABAC_video)==True and check_RBAC_policy((u,r,o),rp,URA)==False):
        print(f'TF{u} {r} {o}')
      elif (check_ABAC_policy((u,r,o),ABAC_video)==False and check_RBAC_policy((u,r,o),rp,URA)==True):
        print(f'FT{u} {r} {o}')
        #number_allow= number_allow+1
      elif (check_ABAC_policy((u,r,o),ABAC_video)==True and check_RBAC_policy((u,r,o),rp,URA)==True):
        correct = correct +1
      elif (check_ABAC_policy((u,r,o),ABAC_video)==False and check_RBAC_policy((u,r,o),rp,URA)==False):
        correct = correct +1
        #number_deny =number_deny +1 
        #print(f'{u}\n{r}\n{o}\n \n')

print(f'For number of correct:{correct}')


For RBAC:
Number of allow:38
Number of deny:118
For ABAC:
Number of allow:38
Number of deny:118
For number of correct:156
